In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
del X_train

In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [25]:
#X_train = pd.read_csv("data/train_inter.csv")
X_test = pd.read_csv("data/test_inter.csv")


In [26]:
#q_train = pd.read_csv("data/freq_train.csv")
q_test = pd.read_csv("data/freq_test.csv")

In [27]:
#X_train = pd.concat([X_train, q_train], axis=1)
X_test = pd.concat([X_test, q_test], axis=1)

In [6]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))


X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())


In [28]:
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [7]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [36]:
del word2vec

In [8]:
def to_vec(x):
    result = word2vec.word_vec('yes') - word2vec.word_vec('yes')
    co = 0
    for i in x:
        if i in word2vec.vocab:
            result += word2vec.word_vec(i)
            co += 1
    return result / co

In [9]:
def norm3(x, y):
    arr = []
    for i in x.index:
        arr += [abs(x.ix[i] - y.ix[i])]
    return arr

In [10]:
def norm2(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [scipy.spatial.distance.cosine(x.ix[i], y.ix[i])]
    return arr

In [11]:
from numpy import linalg as LA
def norm1(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [LA.norm(x.ix[i] - y.ix[i])]
    return arr

In [12]:
tmp1 = X_train['question1'].apply(lambda x: to_vec(x))
tmp2 = X_train['question2'].apply(lambda x: to_vec(x))
tmp1 = pd.DataFrame(list(tmp1))
tmp2 = pd.DataFrame(list(tmp2))
#X_train = pd.concat([X_train, pd.DataFrame(X_train['question1'].apply(lambda x: to_vec(x))),
#                  pd.DataFrame(X_train['question2'].apply(lambda x: to_vec(x)))], axis=1)
from numpy import linalg as LA
#del tmp1
#del tmp2

In [13]:
X_train = pd.concat([X_train, tmp1],axis=1)
X_train = pd.concat([X_train, tmp2],axis=1)

In [14]:
tmp1.fillna(tmp1.mean(), inplace=True)
tmp2.fillna(tmp2.mean(), inplace=True)
1

1

In [15]:
n1 = norm1(tmp1, tmp2)
n2 = norm2(tmp1, tmp2)
#n3 = norm3(tmp1, tmp2)

0
50000
100000
150000
200000
250000
300000
350000
400000
0
50000
100000
150000
200000
250000
300000
350000
400000


In [16]:
n1 = pd.DataFrame(n1)
n1.columns = ['n1']
n2 = pd.DataFrame(n2)
n2.columns = ['n2']
#n3 = pd.DataFrame(n3)
#n3.columns = ['n3']

In [17]:
X_train = pd.concat([X_train, n1],axis=1)
X_train = pd.concat([X_train, n2],axis=1)

In [18]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

#del X_test['question1']
#del X_test['question2']
#del X_test['test_id']

In [20]:
del X_train['n1']
del X_train['n2']

In [20]:
X_train = pd.concat([X_train, n1],axis=1)


In [22]:
X_train = pd.concat([X_train, n2],axis=1)

In [19]:
X_train[:2]

,word_match,tfidf_word_match,q1,q2,0,1,2,3,4,5,...,292,293,294,295,296,297,298,299,n1,n2
0,0.727273,0.772164,1.818125e-07,3.636250e-07,-0.027870,0.026484,0.049194,-0.016388,-0.050432,-0.004822,...,-0.080499,-0.015268,-0.058472,-0.016852,0.026965,0.056458,0.058258,-0.04502,0.238747,0.024187
1,0.307692,0.361758,1.454500e-06,5.454375e-07,0.100639,0.012527,0.066895,0.045807,-0.060059,0.051575,...,-0.050604,0.013800,0.014710,0.007458,-0.011377,-0.032227,0.067801,-0.09425,1.115507,0.455235


In [ ]:
X_train['n1'] - 

In [179]:
X_train.to_csv("data/word2_vec_train.csv", index=None)

In [3]:
X_train = pd.read_csv("data/word2_vec_train.csv")


In [30]:
X_test[:2]

,test_id,question1,question2,word_match,tfidf_word_match,q1,q2
0,0,"[how, does, the, surface, pro, himself, 4, com...","[why, did, microsoft, choose, core, m3, and, n...",0.266667,0.274019,1.818125e-07,1.818125e-07
1,1,"[should, i, have, a, hair, transplant, at, age...","[how, much, cost, does, hair, transplant, requ...",0.500000,0.480962,3.636250e-07,3.636250e-07


In [41]:
del n2

In [45]:
import gc
gc.collect()

7

In [29]:
tmp1 = X_test['question1'].apply(lambda x: to_vec(x))
tmp1 = pd.DataFrame(list(tmp1))

#X_train = pd.concat([X_train, pd.DataFrame(X_train['question1'].apply(lambda x: to_vec(x))),
#                  pd.DataFrame(X_train['question2'].apply(lambda x: to_vec(x)))], axis=1)
#from numpy import linalg as LA
#del tmp1
#del tmp2

In [31]:
X_test = pd.concat([X_test, tmp1],axis=1)

In [35]:
del tmp1

NameError: name 'tmp1' is not defined

In [34]:
tmp2 = X_test['question2'].apply(lambda x: to_vec(x))
tmp2 = pd.DataFrame(list(tmp2))

In [43]:
X_test = pd.concat([X_test, tmp2],axis=1)

In [44]:
del tmp2

In [ ]:
tmp1.fillna(tmp1.mean(), inplace=True)
tmp2.fillna(tmp2.mean(), inplace=True)
1

In [ ]:
n1 = norm1(tmp1, tmp2)

In [ ]:
n2 = norm2(tmp1, tmp2)

In [38]:
n1 = pd.DataFrame(n1)
n1.columns = ['n1']
n2 = pd.DataFrame(n2)
n2.columns = ['n2']

In [40]:
n1.to_csv("data/n1.csv", index=None)
n2.to_csv("data/n2.csv", index=None)

In [46]:
n1 = pd.read_csv("data/n1.csv")
n2 = pd.read_csv("data/n2.csv")

In [47]:
del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [17]:
del X_train

In [ ]:
X_test = pd.concat([X_test, n1],axis=1)
X_test = pd.concat([X_test, n2],axis=1)

In [48]:
X_test['n1'] = n1.values

In [49]:
X_test['n2'] = n2.values

In [50]:
X_test[:3]

,word_match,tfidf_word_match,q1,q2,0,1,2,3,4,5,...,292,293,294,295,296,297,298,299,n1,n2
0,0.266667,0.274019,1.818125e-07,1.818125e-07,0.069040,0.052856,0.065356,0.028613,-0.067700,0.034937,...,-0.090914,0.028786,-0.034929,-0.051687,0.034475,-0.023356,0.061010,-0.048594,0.939924,0.350222
1,0.500000,0.480962,3.636250e-07,3.636250e-07,-0.016091,0.089301,0.072576,0.121937,-0.076483,0.025495,...,-0.151204,0.042806,0.024628,0.166341,0.136393,0.009206,0.027445,-0.027832,0.794613,0.162770
2,0.444444,0.468893,1.818125e-07,1.818125e-07,0.015533,0.035801,0.100991,0.120350,-0.052940,0.021027,...,-0.088867,-0.055176,0.035522,-0.014801,0.108215,-0.110840,-0.020203,-0.074707,0.964996,0.216529


In [45]:
del X_test['n1']
del X_test['n2']

In [51]:
import gc
gc.collect()

21

In [16]:
print(X_test.q1.mean())
print(X_train.q1.mean())


9.668771881958455e-07
9.009205147102744e-07


In [23]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.7,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)
        
        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        
        '''
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred
    
    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.316 +- 0.002


In [53]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [26]:
pred_train = foo(pred_train)

In [27]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_11.csv", index=None)

In [ ]:
1+1

In [24]:
X_train[:10]

,word_match,tfidf_word_match,q1,q2,q1,q2,0,1,2,3,...,290,291,292,293,294,295,296,297,298,299
0,0.727273,0.772164,1.818125e-07,3.636250e-07,1.818125e-07,3.636250e-07,-0.027870,0.026484,0.049194,-0.016388,...,0.016907,0.010545,0.001210,0.000673,0.019775,0.018149,0.013629,0.001577,NaN,NaN
1,0.307692,0.361758,1.454500e-06,5.454375e-07,1.454500e-06,5.454375e-07,0.100639,0.012527,0.066895,0.045807,...,0.069434,0.075851,0.025278,0.016992,0.081116,0.031461,0.081628,0.049377,NaN,NaN
2,0.363636,0.355191,3.636250e-07,1.818125e-07,3.636250e-07,1.818125e-07,0.085183,0.021555,0.063221,0.074796,...,0.000543,0.044193,0.045397,0.022413,0.024248,0.061699,0.042852,0.050974,NaN,NaN
3,0.000000,0.000000,1.818125e-07,1.818125e-07,1.818125e-07,1.818125e-07,-0.039524,0.013211,0.029839,0.106757,...,0.029460,0.107410,0.081034,0.028898,0.074945,0.010297,0.071940,0.036799,NaN,NaN
4,0.000000,0.000000,5.454375e-07,1.818125e-07,5.454375e-07,1.818125e-07,-0.012436,-0.017532,0.095093,-0.013397,...,0.105225,0.079898,0.041982,0.004852,0.126322,0.063192,0.013041,0.011490,NaN,NaN
5,0.470588,0.510771,1.818125e-07,1.818125e-07,1.818125e-07,1.818125e-07,-0.033618,0.046802,0.075024,0.056055,...,0.025364,0.119550,0.021383,0.047138,0.018265,0.079837,0.006089,0.030248,NaN,NaN
6,0.000000,0.000000,1.818125e-07,1.818125e-07,1.818125e-07,1.818125e-07,-0.066569,-0.052165,0.080892,0.247070,...,0.106481,0.104233,0.004899,0.121867,0.141271,0.042704,0.092214,0.059708,NaN,NaN
7,0.500000,0.645836,1.818125e-07,1.818125e-07,1.818125e-07,1.818125e-07,-0.009277,0.000977,0.077466,0.131738,...,0.021360,0.014730,0.015104,0.045239,0.028385,0.039738,0.046163,0.012903,NaN,NaN
8,0.500000,0.396755,3.636250e-07,5.454375e-07,3.636250e-07,5.454375e-07,0.161621,-0.017310,0.104663,0.198584,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
9,0.363636,0.503203,1.818125e-07,1.818125e-07,1.818125e-07,1.818125e-07,0.025894,-0.015422,0.044515,0.148438,...,0.019911,0.022830,0.007516,0.048482,0.014336,0.022736,0.063766,0.002675,NaN,NaN


### Тест

In [56]:
submit = pd.read_csv("data/sample_submission.csv")

In [21]:

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.7,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}


In [22]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=50,
        min_split_gain=0, n_estimators=300, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [54]:
test_pred = lgb.predict_proba(X_test)[:, 1]

In [15]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

In [ ]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)
Xdataval = xgb.DMatrix(data=X_train.values,
                                     label=y_train.values)
Xdatatest = xgb.DMatrix(data=X_test.values)


plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
#bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


#test_pred = bst.predict(Xdatatest)



In [16]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [17]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [20]:
Xdatatest = xgb.DMatrix(data=X_test.ix[:, :].values)


In [21]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [22]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.645145	eval-logloss:0.653016
[15]	train-logloss:0.394068	eval-logloss:0.437614
[30]	train-logloss:0.350926	eval-logloss:0.400341
[45]	train-logloss:0.337777	eval-logloss:0.401014
[60]	train-logloss:0.330209	eval-logloss:0.387795
[75]	train-logloss:0.324804	eval-logloss:0.387937
[90]	train-logloss:0.320158	eval-logloss:0.383231
[105]	train-logloss:0.316191	eval-logloss:0.384437
[120]	train-logloss:0.313145	eval-logloss:0.380075
[135]	train-logloss:0.309691	eval-logloss:0.37662
[150]	train-logloss:0.306753	eval-logloss:0.375377
[165]	train-logloss:0.304356	eval-logloss:0.370649
[180]	train-logloss:0.301772	eval-logloss:0.358448
[195]	train-logloss:0.299199	eval-logloss:0.355894
[210]	train-logloss:0.296768	eval-logloss:0.359679
[225]	train-logloss:0.294945	eval-logloss:0.36081
[240]	train-logloss:0.293169	eval-logloss:0.354961
[255]	train-logloss:0.291001	eval-logloss:0.353774
[270]	train-logloss:0.289022	eval-logloss:0.350644
[285]	train-logloss:0.28755	eval-logloss:

In [23]:
test_pred = bst.predict(Xdatatest)


In [55]:
test_pred = foo(test_pred)

In [57]:
submit['is_duplicate'] = test_pred

In [58]:
submit.to_csv("stacking/lgb_11_test.csv", index=None)

In [59]:
submit

,test_id,is_duplicate
0,0,0.004466
1,1,0.235243
2,2,0.166915
3,3,0.000614
4,4,0.457306
5,5,0.001726
6,6,0.964534
7,7,0.138894
8,8,0.414888
9,9,0.004965
